In [1]:
from __future__ import absolute_import
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import keras
from keras.datasets import mnist
from keras.layers.core import Dense, Flatten, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.utils import np_utils
import matplotlib.pylab as plt
import matplotlib.cm as cm
import numpy as np
np.random.seed(1337)
np.set_printoptions(precision=5, suppress=True)
nb_classes = 10

#image dimensions
img_x, img_y = 28, 28

#load the mnist data-set
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape the data into a 4D tension - (sample numher, x_img_size, y_img_size, num_channles)
# because the MNIST is greyscale, we only have a single channel - RGB colour images would have 3
X_train = X_train.reshape(X_train.shape[0], img_x, img_y, 1)
X_test = X_test.reshape(X_test.shape[0], img_x, img_y, 1)
input_shape = (img_x, img_y, 1)

# convert the data to the right type
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices - this is for use in the
# categorical_crossentropy loss below
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
model1 = Sequential()

convout1 = Conv2D(32, (3,3), strides=(1,1), padding='same', input_shape=input_shape, activation='relu')
model1.add(convout1)
model1.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model1.add(Flatten())
model1.add(Dense(128))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))

model1.add(Dense(nb_classes))
model1.add(Activation('softmax'))

model1.compile(loss='categorical_crossentropy',
             optimizer='adadelta',
             metrics=['accuracy'])

model1.summary()

import os
import h5py
WEIGHTS_FNAME = 'mnist_cnn_weights_v1.hdf'
if False and os.path.exists(WEIGHTS_FNAME):
    # Just change the True to false to force re-training
    print('Loading the existing weights')
    model1.load_weights(WEIGHTS_FNAME)
else:
    batch_size = 128
    nb_epoch = 10
    model1.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
              verbose=1, validation_data=(X_test, Y_test))
    model1.save_weights(WEIGHTS_FNAME, overwrite=True)

score = model1.evaluate(X_test, Y_test, verbose=0)
print('Test Score: ', score[0])
print('Test Accuracy: ', score[1])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               802944    
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)  

KeyboardInterrupt: ignored

In [0]:
#show resulting confusion matrix
np.set_printoptions(precision=3, suppress=True)
print(model1.predict(X_test[1:5]))
print(Y_test[1:5])

Y_pred = model1.predict(X_test)
#Convert one-hot to index
y_pred = np.argmax(Y_pred, axis=1)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [0]:
#Example of a mistaken number using np.where
mis_idx = np.where(y_test != y_pred)[0]
print(mis_idx)
print(mis_idx.shape)
print(list(zip(y_test[mis_idx], y_pred[mis_idx])))

#Visualize a convolved image
from keras import backend as K

inputs = [K.learning_phase()] + model1.inputs

_convout1_f = K.function(inputs, [convout1.output])
def convout1_f(X):
    # the [0] is to disable the training phase flag
    return _convout1_f([0] + [X])

from mpl_toolkits.axes_grid1 import make_axes_locatable

def nice_imshow(ax, data, vmin=None, vmax=None, cmap=None):
    #Wrapper around p1.imshow
    if cmap is None:
        cmap = cm.jet
    if vmin is None:
        vmin = data.min()
    if vmax is None:
        vmax = data.max()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    im = ax.imshow(data, vmin=vmin, vmax=vmax, interpolation='nearest', cmap=cmap)
    plt.colorbar(im, cax=cax)
    
i = mis_idx[0]

#show the first layer of convolutions on an input image
X = X_test[i:i+1]
print(X.shape)
print("target: {}".format(y_test[i]))
print("predicted: {}".format(y_pred[i]))
X.reshape(28, 28)
plt.figure()
plt.title('input')
nice_imshow(plt.gca(), np.squeeze(X), vmin=0, vmax=1, cmap=cm.gray)
plt.show()

In [0]:
import numpy.ma as ma
def make_mosaic(imgs, nrows, ncols, border=1):
    
   #make a mosaic with n-rows and n-columns given set of images
    
    nimgs = imgs.shape[2]
    print(nimgs)
    imshape = imgs.shape[:2]
    print(imshape)
    mosaic = ma.masked_all((nrows * imshape[0] + (nrows - 1) * border,
                           ncols * imshape[1] + (ncols - 1) * border),
                          dtype=np.float32)
    
    paddedh = imshape[0] + border
    paddedw = imshape[1] + border
    for i in range(nimgs):
        row = int(np.floor(i / ncols))
        col = i % ncols
        
        mosaic[row * paddedh:row * paddedh + imshape[0],
              col * paddedw:col * paddedw + imshape[1]] = imgs[:, :, i]
    return mosaic

In [0]:
#show weights
print(len(model1.layers))
print(type(model1.layers[0]))
W = model1.layers[0].get_weights()[0]
W = np.squeeze(W)
print("W shape : ", W.shape)

plt.figure(figsize=(15, 15))
plt.title('conv1 weights')
nice_imshow(plt.gca(), make_mosaic(W, 6, 6), cmap=cm.binary)
plt.show()

In [0]:
#show convolution result
#after activation
C1 = convout1_f(X)
C1 = np.squeeze(C1)
print("C1 shape : ", C1.shape)

plt.figure(figsize=(15, 15))
plt.suptitle('convout1')
nice_imshow(plt.gca(), make_mosaic(C1, 6, 6), cmap=cm.binary)
plt.show()

In [4]:
#initializing the 2nd convolutional layer to use alongside 
#the 1st convolutional layer, which we are recycling from model1
# kernel at 2x2 for better results/accuracy tests
convout2 = Conv2D(32, (2,2), strides=(1,1), padding='same', activation='relu')

model2 = Sequential()

model2.add(convout1)
model2.add(convout2)
model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model2.add(Flatten())
model2.add(Dense(128))
model2.add(Activation('relu'))
model2.add(Dropout(0.5))

model2.add(Dense(nb_classes))
model2.add(Activation('softmax'))

model2.compile(loss='categorical_crossentropy',
             optimizer='adadelta',
             metrics=['accuracy'])

model2.summary() 

WEIGHTS_FNAME2 = 'mnist_cnn_weights_v2.hdf'
if False and os.path.exists(WEIGHTS_FNAME2):
    # Just change the True to false to force re-training
    print('Loading the existing weights')
    model2.load_weights(WEIGHTS_FNAME2)
else:
    batch_size = 128
    nb_epoch = 10
    model2.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
              verbose=1, validation_data=(X_test, Y_test))
    model2.save_weights(WEIGHTS_FNAME2, overwrite=True)

score = model2.evaluate(X_test, Y_test, verbose=0)
print('Test Score: ', score[0])
print('Test Accuracy: ', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        4128      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               802944    
_________________________________________________________________
activation_5 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

KeyboardInterrupt: ignored

In [0]:
#Time to view the resulting confusion matrix
np.set_printoptions(precision=3, suppress=True)
print(model2.predict(X_test[1:5]))
print(Y_test[1:5])

Y_pred = model2.predict(X_test)
#Convert one-hot to index
y_pred = np.argmax(Y_pred, axis=1)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [0]:
#Example of a mistaken number using np.where
mis_idx = np.where(y_test != y_pred)[0]
print(mis_idx)
print(mis_idx.shape)
print(list(zip(y_test[mis_idx], y_pred[mis_idx])))

#Visualize a convolved image
inputs = [K.learning_phase()] + model2.inputs

_convout2_f = K.function(inputs, [convout2.output])
def convout2_f(X):
    #the [0] is to disable the training phase flag
    return _convout2_f([0] + [X])
    
i = mis_idx[0]

#show first layer of convolutions on an input image
X = X_test[i:i+1]
print(X.shape)
print("target: {}".format(y_test[i]))
print("predicted: {}".format(y_pred[i]))
X.reshape(28, 28)
plt.figure()
plt.title('input')
nice_imshow(plt.gca(), np.squeeze(X), vmin=0, vmax=1, cmap=cm.gray)
plt.show()

In [0]:
print(len(model2.layers))
print(type(model2.layers[0]))
W = model2.layers[0].get_weights()[0]
W = np.squeeze(W)
print("W shape : ", W.shape)

plt.figure(figsize=(15, 15))
plt.title('conv2 weights')
nice_imshow(plt.gca(), make_mosaic(W, 6, 6), cmap=cm.binary)
plt.show()

In [0]:
#show convolution result 
#after activation
C1 = convout2_f(X)
C1 = np.squeeze(C1)
print("C1 shape : ", C1.shape)

plt.figure(figsize=(15, 15))
plt.suptitle('convout2')
nice_imshow(plt.gca(), make_mosaic(C1, 6, 6), cmap=cm.binary)
plt.show()

Questions :


How many hidden layers are in your network? 

In my second  model I included  2 Convolutional layers, 1 hidden layer, one fully connected layer and one flattening layer. 5 total hidden layers.

How many convolutions are calculated in each convolution layer?

In my convolutional neural network I use padding. I use padding because when we take our 3 by 3 kernel for our first model snd with 9 total pixels, if the center of our kernel(center pixel) is close enough to the edge then that means that part of our kernel will hang off of the picture. By specifying the padding the padding allows us to let our kernel hang off the edge by only one space in a certain direction and one space in the other direction. If each image is 28 by 28 then the total number of pixels comes to 784. In order to account for the extra row we will need  1 by 1 padding on all sides. We will multiply the length of a row of pixels by 4 and then add 4. We will add 4 for the 4 off-grid pixels in the corner. The result area will be  784 + 4 + 28 * 4. Result of 900 total. Our convolutions can hang off of the edge by 1 pixel and our strides are 1x1, a centered convolution will occur in all 84 spaces within the 28 by 28 image. 784 convolutions will be calculated in each layer.

Run a single test example through the model and print some of the convolved images from the first layer. Can you see any features from the image that are revealed by printing? 

Yes, the printed images have various shades of the number that we try to predicted. I noticed that the shades of the number vary on the spectrum from the inner portion of the number to the outer edges. We also notice that with this, some parts of the image are more focused on than others.

Print a couple of the convolution kernels as matrices (no need to print images). What kind of patterns can you see in the convolution kernels? 

The convolution kernels seem to perform the same way, except the kernel demensions are different from model 1 and 2. when i ran model 2 with 2x2 instead of 3x3 i seemed to get better results/accuracy. The matrices seem to be the same general pattern with a few numbers that different in outlining areas. 